# spine instance by unet - 2D

In [1]:
# import library
import sys
import tifffile
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from skimage.io import imsave
from skimage.morphology import remove_small_objects

InteractiveShell.ast_node_interactivity = 'last_expr'

from tqdm import tqdm

sys.path.append("..")
sys.path.append("..//..")
import os

import numpy as np
#load model
import tensorflow as tf
from IPython.core.interactiveshell import InteractiveShell
from skimage.filters import threshold_otsu, threshold_yen
from skimage.filters.thresholding import (threshold_isodata, threshold_li,
                                          threshold_local, threshold_mean,
                                          threshold_minimum,
                                          threshold_multiotsu,
                                          threshold_niblack, threshold_otsu,
                                          threshold_triangle, threshold_yen)
from skimage.io import imsave
from skimage.morphology import remove_small_objects
from skimage.segmentation import expand_labels, watershed

import spinelib
from spinelib import cflow, imgio, seg
from spinelib.cflow.blob import find_spheres, peakfilter, sphere_log
from spinelib.cflow.localthreadhold import local_threshold
from spinelib.imgio import napari_base
from spinelib.seg import segment, track
from spinelib.utils import npixel
from spinelib.utils.measure import label_statics
from spinelib.utils.spine_struct import spines_distance
from utils import file_base

InteractiveShell.ast_node_interactivity = 'last_expr'
methods = cflow.localthreadhold.thmethods

c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


No Fortran modules found, falling back on Python implementation.
Did you run `python3 setup.py install`?


In [2]:
#basic fun

def show_img_info(images):
    shape=images.shape
    dtype=images.dtype
    ndim=images.ndim
    print("image shape :",shape)
    print("image dtype :",dtype)
    print("image ndim :",ndim)


def remove_den(imgs,unetmodel,cval=140):
    denmasks,_=predict.predict_time_img(unetmodel,imgs)
    imgs=imgs.copy()
    imgs[denmasks==1]=cval
    return imgs
def remove_small_lable(mask,thsize):
    labs=measure.unique_labs(mask)
    newmask=np.zeros_like(mask)
    for lab in labs:
        mm=mask==lab
        mask2=remove_small_objects(mm,thsize)
        if not mask.any():
            mask2=mm
        newmask[mask2]=lab
    return newmask   
def th_img(imgs,ls,binary_func):
    global_th=binary_func(imgs[0])
    lss=np.zeros_like(imgs,dtype=np.uint16)

    for i in range(len(imgs)):
        img=imgs[i]
        adth=img>global_th
        # denmask,_=predict.predict_single_img(unetmodel,img)
        l=ls*adth
        l[l<2]=0
        l=remove_small_lable(l,6)
        lss[i]=l
        # lss[i][denmask==1]=0
    lss[lss<2]=0
       
    return lss

def modify_masks(masks,sizeth=4):
    for n,mask in enumerate(masks):
        masks[n]=modify_mask(mask,sizeth=sizeth)
def modify_mask(mask,sizeth=4):
    denmask=mask==1
    spinemask=mask==2
    denmask=remove_small_objects(denmask,min_size=sizeth)
    spinemask=remove_small_objects(spinemask,min_size=sizeth)
    mask=spinemask*2+denmask
    return mask

In [3]:
# super params
#-----------------------#
#   paras  #
#-----------------------#

steps=2 # caculate stick gradient step lenth 1-4, larger more error , smaller than spine diameter
sigma=(2,2) # z,y,x
lambda1=1
lambda2=1.5 # lambda2 need large than lambda1
sigma_hessian = 4
sigma_supp = 100
countth=10*20 # fiber largest pixels, include z stack, that to say lenth*stackn
smallobj=200
spine_radius=35 # for iteration to growth area
maxspinesize=200
spotrange=[3,11]
binary_func=local_threshold#threshold_li
use_sobel=False
search_box=[25,25]
keep_time=5
peak_use_gaussian=True

untmodel=None

In [4]:
# load data
#-----------------------#
#   data loader  #
#-----------------------#
filename=r"D:\data\Train\QL\2Dt\Stablized cLTP 20150921n1.tif"
#-----------------------#
#   Output file path  #
#-----------------------#
"""
statistic

remove_den
    spine img
roi dir
    - each frame roi
    - max fram roi
predict dir 
    - mask 0,1,2
    - spine pr
    - den pr
"""

ddir,shotname,suffix=file_base.split_filename(filename)
savedir=r"D:\data\Train\QL\2Dt\statistic"
dendir=r"D:\data\Train\QL\2Dt\remove_den"
roidir=r"D:\data\Train\QL\2Dt\roi_proj"

predictdir=r"D:\data\Train\QL\2Dt\predict"

areadir=os.path.join(savedir,"area")
meandir=os.path.join(savedir,"mean")
file_base.create_dir(areadir)
file_base.create_dir(meandir)
file_base.create_dir(dendir)
file_base.create_dir(roidir)
file_base.create_dir(predictdir)


areafile=os.path.join(areadir,shotname+"_area.csv")
meanfile=os.path.join(meandir,shotname+"_mean_inten.csv")
sumfile=os.path.join(meandir,shotname+"_sum_inten.csv")

removedenfile=os.path.join(dendir,shotname+"_remove_den.tif")
skeletonfile=os.path.join(dendir,shotname+"_skeleton_den.tif")

proj_roifile=os.path.join(roidir,shotname+"_roi_proj.tif")
each_roifile=os.path.join(roidir,shotname+"_roi.tif")

maskfile=os.path.join(predictdir,shotname+"_mask.tif")
spineprfile=os.path.join(predictdir,shotname+"_spine_pr.tif")
denprfile=os.path.join(predictdir,shotname+"_den_pr.tif")



areafile=os.path.join(areadir,shotname+"_area.csv")
meanfile=os.path.join(meandir,shotname+"_mean_inten.csv")
denfile=os.path.join(dendir,shotname+"_removeden.tif")
#-----------------------#
#   model loader  #
#-----------------------#


#---  dendrite filter model---#
from networks import unet
unetmodel=unet.UNet2D()
modelpath=r"D:\code\myspine\models\M2d_seg\modelep200-loss0.133.h5"
modelpath=r"D:/code/myspine/modelsQL/M2d_seg\modelep006-loss0.097.h5"
unetmodel.build(input_shape =(4,512,512,1))
unetmodel.load_weights(modelpath)


imgs = imgio.ski_imread(filename)
show_img_info(imgs)
# imgs=imgs[:10]

ModuleNotFoundError: No module named 'networks'

# PART ONE
track by max projection ,maybe stable


1. Let's see one img

In [7]:
#--- BINARY---#
img=np.max(imgs,axis=0)

adth=binary_func(img)
#adth=img>global_th
# adth=local_threshold(imgs[0],offset=100)
noise_mean=np.mean(img[~adth])


#--- PEAK SPOT FILTER : SPINE SEED---#
mask,spinepr,_=unetmodel.predict_single_img(img)
mask=modify_mask(mask,sizeth=10)
pr_corner=peakfilter(spinepr,3,0,use_gaussian=True)*(spinepr>0.3)#*adth
spine_label=segment.label_instance_water(spinepr,pr_corner,spinepr>0.3, 
                                 maxspinesize,searchbox=[7,7])
spine_label=remove_small_objects(spine_label,min_size=10)
# filter seed
points2=np.array(napari_base.get_point(pr_corner))

viewer=imgio.plot.imshow3d(I=img,Is_overlay=[spinepr],name_Is=["img","spinepr"],
                           label_lay=[spine_label,mask],label_lay_name=["spine label","mask"],
                           point_lay=[points2,]
                           )

1/1 [==============================] - 1s 557ms/step


..\networks\unet.py:320: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return imgns[obj],spineprs[obj],denprs[obj]
..\spinelib\utils\npixel.py:91: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return arr[obj]


In [68]:
spine_labels=[]
spineprs=[]
masks=[]
masks,spineprs,_=unetmodel.predict_single_img(imgs)
# for img in imgs:
#     #--- BINARY---#

#     adth=binary_func(img)
#     #adth=img>global_th
#     # adth=local_threshold(imgs[0],offset=100)
#     noise_mean=np.mean(img[~adth])


#     #--- PEAK SPOT FILTER : SPINE SEED---#
#     mask,spinepr,_=unetmodel.predict_single_img(img)
#     mask=modify_mask(mask,sizeth=10)
#     pr_corner=peakfilter(spinepr,3,0,use_gaussian=True)*(mask==2)#*adth
#     spine_label=segment.label_instance_water(spinepr,pr_corner,spinepr>0.4, 
#                                     maxspinesize,searchbox=[7,7])

#     # filter seed
#     #points2=np.array(napari_base.get_point(pr_corner))
#     spine_labels.append(spine_label)
#     spineprs.append(spinepr)
#     masks.append(mask)
    
# spine_labels=np.array(spine_labels)
# masks=np.array(masks)
# spineprs=np.array(spineprs)
viewer=imgio.plot.imshow3d(I=imgs,Is_overlay=[spineprs],name_Is=["spinepr"],
                        label_lay=[spine_labels,masks],label_lay_name=["spine label","mask"],
                        
                        )



1/1 [==============================] - 0s 306ms/step


KeyboardInterrupt: 

In [123]:
aaa=spine_label.copy()

In [11]:
# see and edit result
# spine_label=aaa.copy()
viewer=imgio.plot.imshow3d(I=imgs,Is_overlay=[spinepr,spinepr],name_Is=["img","spinepr"],
                           label_lay=[spine_label],label_lay_name=["spine"],
                           )

edit_spine_label_proj=viewer.layers["spine"].data


2. Then see all time series

In [8]:
# precit all frame
all_mask,all_spine_pr,all_den_pr= unetmodel.predict_time_imgs(imgs)
modify_masks(all_mask,sizeth=4)



Processing:   0%|          | 0/80 [00:00<?, ?it/s]

1/1 [==============================] - 0s 275ms/step


Processing:   1%|▏         | 1/80 [00:00<00:24,  3.24it/s]

1/1 [==============================] - 0s 305ms/step


Processing:   2%|▎         | 2/80 [00:00<00:25,  3.07it/s]

...

Processing:  98%|█████████▊| 78/80 [00:24<00:00,  3.29it/s]

1/1 [==============================] - 0s 269ms/step


Processing:  99%|█████████▉| 79/80 [00:24<00:00,  3.30it/s]

1/1 [==============================] - 0s 274ms/step


Processing: 100%|██████████| 80/80 [00:24<00:00,  3.25it/s]


In [9]:
binary_func=threshold_isodata
def adth_func_time(imgs,binary_func):
    if binary_func.__name__=='local_threshold':
        adths=[binary_func(img) for img in imgs]
        ads=np.array(adths)
        meanbg = np.mean(imgs[ads<1])
        th=np.max(imgs[ads<1])
    else:    
        mim=np.mean(imgs[0:5],axis=0)
        th=binary_func(mim)
        meanbg = np.mean(mim[mim<th])
            
        adths=[img>th for img in imgs]
    return np.array(adths),th,meanbg
all_adths,th,meanbg=adth_func_time(imgs,binary_func)

In [14]:

proj_spinepr=np.max(all_spine_pr,axis=0)
ppr=np.mean(all_spine_pr,axis=0)
proj_mask=np.max(all_mask,axis=0)
proj_adths=np.max(all_adths,axis=0)
pr_corner=peakfilter(ppr,3,0,use_gaussian=True)*(proj_spinepr>0.7)*proj_adths
proj_spine_label=segment.label_instance_water(proj_spinepr,pr_corner,(proj_spinepr>0.7), 
                                 maxspinesize,searchbox=[7,7])

proj_spine_label=remove_small_objects(proj_spine_label,min_size=20)
viewer=imgio.plot.imshow3d(I=imgs,Is_overlay=[proj_spinepr,proj_adths],name_Is=["spinepr","adths"],
                           label_lay=[proj_spine_label,spine_label],label_lay_name=["spine label","mask"],
                           #point_lay=[points2,]
                           )

In [16]:
# see and edit result

viewer=imgio.plot.imshow3d(I=imgs,Is_overlay=[spinepr],name_Is=["spinepr"],
                           label_lay=[proj_spine_label],label_lay_name=["spine"],
                           )

edit_spine_label_proj=viewer.layers["spine"].data

In [125]:
aaa=edit_spine_label_proj.copy()

In [16]:
# view multi frames segment
all_spine_imgs=segment.keep_spineimg_bypr(imgs,all_mask,all_spine_pr,th=0.5)#keep_spine(imgs,all_mask)


# t_edit_pr_spine_proj=edit_spine_label_proj


t_edit_pr_spine_proj,_=segment.resortseg(t_edit_pr_spine_proj,2)
proj_spine_masks=np.array([t_edit_pr_spine_proj for im in imgs])
all_spine_maskes=(all_spine_pr>0.1)*proj_spine_masks*(np.max(all_spine_pr,axis=0)>0.1)#*(all_mask==2)*all_adths
"""illustration
imgs
"spine prob" : predict spine probability
"adth img spine" : img of spine area , masked by threshold mask and spine mask
"all segmnet" : predict classification result
"""

viewer=imgio.plot.imshow3d(imgs,
                              [all_spine_pr,all_spine_imgs*all_adths,all_adths],
                              name_Is=["spine prob","spine img","adths"],
                              label_lay=[all_spine_maskes,t_edit_pr_spine_proj],
                              label_lay_name=["spine label","proj spine label"])

3. save mask

In [18]:
# save result

#save roi mask


#edit_pr_spine_proj_t : spine max mask (same size for each frame)

# imsave(maskfile,all_mask)
# imsave(spineprfile,all_spine_pr)

# all_mask : den spine back mask 0 1 2
tifffile.imwrite(
    maskfile,
    all_mask.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TYX'},
)

# all_spine_pr : spine probability
tifffile.imwrite(
    spineprfile,
    all_spine_pr,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TYX'},
)


tifffile.imwrite(
    denprfile,
    all_den_pr,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TYX'},
)

# max frame spine mask
imsave(proj_roifile,t_edit_pr_spine_proj.astype(np.uint16))
# nspine_maskes : spine's area mask
tifffile.imwrite(
    each_roifile,
    all_spine_maskes.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TYX'},
)

# removedenfile : remove den area imgs
tifffile.imwrite(
    removedenfile,
    all_spine_imgs,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TYX'},
)

D:\data\Train\QL\2Dt\roi_proj\AVG_seq0_StackedSlices-SIM561_RedCh_SIrecon-2_roi_proj.tif is a low contrast image


# Part Two 

1. load data

In [181]:

all_mask=imgio.ski_imread(maskfile)
all_spine_pr=imgio.ski_imread(spineprfile)
all_den_pr=imgio.ski_imread(denprfile)

same_spine_maskes=imgio.ski_imread(proj_roifile)
spine_maskes=imgio.ski_imread(each_roifile)

spine_imgs=imgio.ski_imread(removedenfile)

In [182]:
binary_func=threshold_isodata
def adth_func_time(imgs,binary_func):
    if binary_func.__name__=='local_threshold':
        adths=[binary_func(img) for img in imgs]
        ads=np.array(adths)
        meanbg = np.mean(imgs[ads<1])
        th=np.max(imgs[ads<1])
    else:    
        mim=np.mean(imgs[0:5],axis=0)
        th=binary_func(mim)
        meanbg = np.mean(mim[mim<th])
            
        adths=[img>th for img in imgs]
    return np.array(adths),th,meanbg
all_adths,th,meanbg=adth_func_time(imgs,binary_func)

In [186]:
spine_maskes=[]
for spinepr,adth in zip(all_spine_pr,all_adths):
    spine_maskes.append(same_spine_maskes*(spinepr>0.3)*adth)
   
spine_maskes=np.array(spine_maskes)
viewer=imgio.plot.imshow3d(imgs,
                              [all_spine_pr,all_den_pr,all_adths],
                              name_Is=["spine prob","denpr","adth"],
                              label_lay=[spine_maskes,same_spine_maskes],
                              label_lay_name=["spine label","proj spine label"])

2. caculate statistic

In [187]:
import pandas as pd
from spinelib.utils import measure
from skimage.measure import label, regionprops, regionprops_table
same_spine_maskes_t=np.array([same_spine_maskes])

same_spine_maskes_ts=np.array([same_spine_maskes for im in imgs])


arr=measure.label_series_statics(imgs,spine_maskes,measure="area")
df_volume=arr*0.16*0.16
df_volume=pd.DataFrame(df_volume)
df_volume.rename(columns={0:'bg',1:'den'},inplace=True)

#area
arr=measure.label_series_statics(same_spine_maskes_t,same_spine_maskes_t,measure="area")
arr_volume_same=arr
#inten
arr=measure.label_series_statics(spine_imgs,same_spine_maskes_ts,measure="intensity_mean")
backvs=[np.mean(im[ms==0]) for im,ms in zip(imgs,all_mask)]
#bg
arr=np.array(arr)
arr[:,0]=backvs
arr=np.row_stack((arr_volume_same,arr))
df_mean=pd.DataFrame(arr)
df_mean.rename(columns={0:'bg',1:'den'},index={0:"area"},inplace=True)



shape (85, 512, 512) (85, 512, 512)
shape (1, 512, 512) (1, 512, 512)
shape (85, 512, 512) (85, 512, 512)


In [185]:
# save csv result
df_volume.to_csv(areafile)
df_mean.to_csv(meanfile)

# 3. Part three ： spine to dendrite

1.modify den skeleton

In [90]:
#--- BINARY---#
img=np.max(imgs,axis=0)
# img=morphology.white_tophat(img,morphology.cube(5))
global_th=binary_func(img)
adth=img>global_th
# adth=local_threshold(imgs[0],offset=100)
noise_mean=np.mean(img[~adth])
img=img-noise_mean
img[img<0]=0
img.shape
all_mask=imgio.ski_imread(maskfile)
from skimage.morphology import skeletonize_3d
den=skeletonize_3d(np.max(all_mask==1,axis=0))
den=den>0
structure=np.ones((5,)*2)
den=morphology.binary_dilation(den,structure)
den=den=skeletonize_3d(den)
viewer=imgio.plot.imshow3d(img,label_lay=[den],
                           label_lay_name=["den"],
                                 )

Exception in callback BaseAsyncIOLoop._handle_events(19228, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(19228, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [91]:
viewer=imgio.plot.imshow3d(img,label_lay=[den],
                           label_lay_name=["den"],
                                 )
den=viewer.layers["den"].data

Exception in callback BaseAsyncIOLoop._handle_events(17516, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(17516, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [92]:
import scipy.ndimage as ndi

from skimage import morphology
den=den>0
structure=np.ones((3,)*2)
den=morphology.binary_dilation(den,structure)
den=den=skeletonize_3d(den)
def removenode(skeleton,connection):
    for i in range(1,skeleton.shape[0]-1):
        for j in range(1,skeleton.shape[1]-1):
            if cflow.skeleton_leaf.get_border_connection(skeleton,i,j)>=connection:
                skeleton[i,j]=0
removenode(den,3)
den,_=ndi.label(den,np.ones((3,)*2))
viewer=imgio.plot.imshow3d(img,label_lay=[den],
                           label_lay_name=["den"],
                                 )

Exception in callback BaseAsyncIOLoop._handle_events(20108, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(20108, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

2. save dendrite skeleton

In [93]:
tifffile.imwrite(
    skeletonfile,
    den.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'YX'},
)


3. get trace from spine to dendrite

In [5]:
#--- BINARY---#
all_mask=imgio.ski_imread(maskfile)
den=imgio.ski_imread(skeletonfile)
img=np.max(imgs,axis=0)
global_th=binary_func(img)
adth=img>global_th
# adth=local_threshold(imgs[0],offset=100)
noise_mean=np.mean(img[~adth])
same_spine_maskes=imgio.ski_imread(proj_roifile)
same_spine_maskes[den>0]=1
cords,linemask=spines_distance(img,same_spine_maskes,[20,20])


..\spinelib\utils\spine_struct.py:191: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropimg=img[bindbox].copy()
..\spinelib\utils\spine_struct.py:192: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  croplable=labels[bindbox]
..\spinelib\utils\spine_struct.py:210: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  linemas

In [8]:
pointers=[cord[0] for k,cord in cords.items() if cord]
# print(pointers)
vectors=[(cord[0],np.array(cord[1])-cord[0]) for k,cord in cords.items() if cord[1]]
viewer=imgio.plot.imshow3d(img,label_lay=[all_mask,linemask,den],
                           label_lay_name=["label","line","den"],
                           point_lay=[pointers]
                                 )
viewer.add_vectors(
            vectors
        )

<Vectors layer 'Vectors' at 0x2b522d41bb0>

Exception in callback BaseAsyncIOLoop._handle_events(8540, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(8540, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData\R

4. caculate density

In [96]:
import collections
cnts=collections.defaultdict(int)
for k,cord in cords.items():
    if cord[1]:
        x,y=cord[1]
        cnts[den[x,y]]+=1 

dlabels,dareas,indexs=spinelib.utils.measure.label_statics(den)
for lab,area in zip(dlabels,dareas):
    print(lab,area*0.16,cnts[lab],cnts[lab]/area/0.16)

1 72.8 95 1.304945054945055
3 75.2 94 1.25
5 40.160000000000004 44 1.0956175298804782


# part 4 : caculate local cluster corelation

In [47]:


#--- PEAK SPOT FILTER : SPINE SEED---#
# %matplotlib inline
denmask,_=predict.predict_single_img(unetmodel,img)
corner=peakfilter(img,3,0,use_gaussian=True)*adth*(denmask!=1)


# filter seed
points2=np.array(napari_base.get_point(corner))
points,points2=model_predict(points2,img,peakmodel,boxsize=(16,16))

#--- GROW AREA---#
corner2=napari_base.get_mask_from_point(points,img.shape)
labels,num=segment.ndilable(corner2,1)
ls = segment.foreach_grow(img, num_iter=spine_radius, 
                              init_level_set=labels,
                              searchbox=search_box,
                              sizeth=maxspinesize,adth=adth,
                              method="geo",smoothing=0)
mask=(denmask==1).astype(np.uint16)
mask[mask<1]=ls[mask<1]
mask=watershed(-img,mask,mask=adth)
mask=remove_small_objects(mask,4)
mask,_=segment.resortseg(mask)
# ls=segment.fill_hulls(ls)
# np.save("kkk.npy",dspimg)
# points2=filter_p3d(points,img,clff)
ls=mask*(mask>1)

In [48]:
viewer=etsynseg.plot.imshow3d(np.concatenate((img[None,...],imgs),axis=0),[mask], label_lay=[ls],lablel_lay_name=["lls"])
ls2=viewer.layers["lls"].data
mask[ls2>0]=ls2[ls2>0]

Exception in callback BaseAsyncIOLoop._handle_events(12104, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(12104, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [146]:
# mask,_=segment.resortseg(mask)
lss=th_img(imgs,mask,binary_func)
lss,_=segment.resortseg(lss,2)

In [147]:
viewer=etsynseg.plot.imshow3d(imgs, label_lay=[np.array([lss]),ls],lablel_lay_name=["lls","ls"])

Exception in callback BaseAsyncIOLoop._handle_events(23844, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(23844, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [148]:
from utils import measure
all_lss=[ls for img in imgs]
df=measure.label_series_statics(imgs,all_lss,measure="mean_intensity")
df2=measure.label_series_statics(imgs,lss,measure="area")

In [12]:
df2.to_csv("area.csv")

In [13]:
viewer=etsynseg.plot.imshow3d(imgs,[adth], label_lay=[all_lls,lss],lablel_lay_name=["all","lss"])
# labels = viewer.layers["Labels"].data


In [14]:
# run time


# =========initial series =======
corners=np.zeros_like(imgs)
nofinders=None
labcounts={}
oimg=imgs[0]
# ols=ls
ols=None
startl=1
# =========time series store=======
ims=[]
lss=[]
lss2=[]
peaks=[]
tracks=[]
adths=[]
miss=[]
# for img in imgs:
#     adth=local_threshold(img)

nz=imgs.shape[0]
objects=[]
for i in tqdm(range(len(imgs)), desc='Processing'):
    img=imgs[i]
    adth=img>global_th
    corner=peakfilter(img,3,0,use_gaussian=peak_use_gaussian)*adth
    ls=all_lls*adth
    
    lss.append(ls)
    ims.append(img)
    peaks.append(corner)
    # miss.append(nofinders)
    # adths.append(adth)
    # ols=nls2

Exception in callback BaseAsyncIOLoop._handle_events(9068, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(9068, 1)>
Traceback (most recent call last):
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData\R

In [15]:
viewer=etsynseg.plot.imshow3d(imgs, [peaks],label_lay=[np.array([lss])])

IndexError: tuple index out of range

Exception in callback BaseAsyncIOLoop._handle_events(9628, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(9628, 1)>
Traceback (most recent call last):
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData\R

# PART2
> tracking by each frame , not stable

In [17]:
# run time

corners=np.zeros_like(imgs)
# lss=np.zeros_like(imgs)

# =========initial series =======
nofinders=None
labcounts={}
oimg=imgs[0]
# ols=ls
ols=None
startl=1
# =========time series store=======
ims=[]
lss=[]
lss2=[]
peaks=[]
tracks=[]
adths=[]
miss=[]
# for img in imgs:
#     adth=local_threshold(img)
global_th=binary_func(imgs[0])
nz=imgs.shape[0]
objects=[]

for i in tqdm(range(len(imgs)), desc='Processing'):
    # startl , labcount, nofinder,ols will be update
    img=imgs[i]
    adth=img>global_th
    #adth=local_threshold_3d(imgs[0],method="mean")
    # noise_mean=np.mean(img[~adth])
    # img=img-noise_mean
    # img[img<0]=0
    #adth=local_threshold(img)
    # adth=remove_small_objects(adth,2)
    corner=peakfilter(img,3,0,use_gaussian=peak_use_gaussian)*adth
    points0=np.array(napari_base.get_point(corner))
    points,points2=model_predict(points0,img,lmodel,boxsize=(16,16))
    
    corner2=napari_base.get_mask_from_point(points,img.shape)
    nls,nofinders,updatalabs,_peak=track.track_by_seg(img,ols,corner,adth,
                               spine_radius,labcounts=labcounts,
                               resvers=nofinders,
                               searchbox=search_box,keep_time=keep_time,
                              sizeth=maxspinesize,
                              method="geo",smoothing=0)
    track.label_pre(lss2,updatalabs,nls,i)
    corner3=corner2>nls
    labels,num=ndi.label(corner3)
    labels[labels>0]+=startl
    nls2 = segment.foreach_grow(img, num_iter=spine_radius, 
                              init_level_set=labels,
                              searchbox=search_box,
                              sizeth=maxspinesize,adth=adth,
                              method="geo",smoothing=0,oldseg=nls)
    nls2,startl=segment.resortseg_truncate(nls2,start=startl)
    
    # nls,peak,nofinders=track_by_seg(img,ols,3,spine_radius,labcounts=labcounts,resvers=nofinders,
    #                            searchbox=[5,25,25],keep_time=5,
    #                           sizeth=maxspinesize,
    #                           method="geo",smoothing=0)
    lss.append(nls)
    lss2.append(nls2)
    ims.append(img)
    peaks.append(_peak)
    miss.append(nofinders)
    adths.append(adth)
    ols=nls2


Processing: 100%|██████████| 46/46 [00:07<00:00,  6.08it/s]


In [18]:
from utils import measure
def _trunc_label(ls,labs):
    mask=np.zeros_like(ls)
    for lab in labs:
        mm=ls==lab
        mask[mm]=lab
    return mask

def _merge_label(ls1,ls2):
    #ls2 add to ls1
    def most_labs(vs,lab):
        ar_unique, c = np.unique(vs, return_counts=True)
        arr1inds = c.argsort()
        sorted_c = c[arr1inds[::-1]]
        sorted_lab = ar_unique[arr1inds[::-1]]
        clab,c=0,0
        for clab,c in zip(sorted_c,sorted_lab):
            if clab==0:continue
            if clab==lab:continue
            return clab,c
        return clab,c
        
    labs=measure.unique_labs(ls2)

    lab_dict=[]
    for lab in labs:
        mm=ls2==lab
        c2=np.sum(mm)
        vs=ls1[mm]
        clab,c=most_labs(vs,lab)
        if c>0.1*c2:
            lab_dict.append((clab,lab))
            
            # ls1[ls1==clab]=lab
            # ls1[mm]=lab
        else:
            
            ls1[mm]=lab
    return ls1,lab_dict
            
        

def reverse_track_by_seg(imgs,lss,adths,peaks,num_iter,searchbox,keep_time=3,resvers=None,labcounts={},
                sizeth=np.inf,method="geo",
                smoothing=0,lambda1=1, lambda2=3):
    #assume spine not move quickly
    # reverse match
    imgl=len(lss)
    for i1 in range(imgl-1,0,-1):
        ls1=lss[i1]
        ls2=lss[i1-1]
        prelab=set(np.unique(ls1))
        curlab=set(np.unique(ls2))
        diflab=prelab-curlab
        peakmask=peaks[i1-1]
        ls=_trunc_label(ls1,diflab)
        img=imgs[i1-1]
        adth=adths[i1-1]
        hitpeak=(ls*peakmask).astype(np.uint64)
        intersecls=segment.foreach_grow(img,num_iter,hitpeak,
                     searchbox=searchbox,sizeth=sizeth,
                     adth=adth,method=method,smoothing=smoothing,
                     lambda1=lambda1,lambda2=lambda2)
        ls,_=_merge_label(ls2,intersecls)
        lss[i1-1]=ls

In [23]:
reverse_track_by_seg(ims,lss2,adths,peaks,num_iter=spine_radius,searchbox=search_box,keep_time=3,resvers=None,labcounts={},
                sizeth=maxspinesize,method="geo",
                smoothing=0,lambda1=1, lambda2=3)

In [24]:
viewer=etsynseg.plot.imshow3d(np.array(ims), [peaks,miss],label_lay=[np.array(lss2,dtype=np.uint64),np.array(lss,dtype=np.uint64)],lablel_lay_name=["lss","ls"])

Exception in callback BaseAsyncIOLoop._handle_events(15792, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(15792, 1)>
Traceback (most recent call last):
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [27]:
#brack
configfile=r"D:\spine\segment\some_analysis_for_spine_img\models\cell_config1.json"
newlls=track.track_all_series(imgs,lss2,peaks,configfile,search_radius=5,tracklength=10)

[INFO][2022/05/07 01:36:40 PM] btrack (v0.4.3) library imported
[INFO][2022/05/07 01:36:40 PM] Setting max XYZ search radius to: 100
[INFO][2022/05/07 01:36:40 PM] Starting BayesianTracker session
[INFO][2022/05/07 01:36:40 PM] Loading motion model: cell_motion
[INFO][2022/05/07 01:36:40 PM] Setting max XYZ search radius to: 5
[INFO][2022/05/07 01:36:40 PM] Set volume to ((0, 1200), (0, 1200), (0, 1200))
[INFO][2022/05/07 01:36:40 PM] Starting tracking... 
[INFO][2022/05/07 01:36:40 PM] Tracking objects in frames 0 to 10 (of 10)...


2
ndim 2


[INFO][2022/05/07 01:36:41 PM] SUCCESS.
[INFO][2022/05/07 01:36:41 PM]  - Found 969 tracks in 10 frames (in 0.0s)
[INFO][2022/05/07 01:36:41 PM]  - Inserted 44 dummy objects to fill tracking gaps
[INFO][2022/05/07 01:36:41 PM] Loading hypothesis model: cell_hypothesis
[INFO][2022/05/07 01:36:41 PM] Calculating hypotheses (relax: True)...
[INFO][2022/05/07 01:36:41 PM]  - Fates.FALSE_POSITIVE: 969 (of 969)
[INFO][2022/05/07 01:36:41 PM]  - Fates.LINK: 0 (of 312)
[INFO][2022/05/07 01:36:41 PM]  - Fates.INITIALIZE_BORDER: 0 (of 969)
[INFO][2022/05/07 01:36:41 PM]  - TOTAL: 2250 hypotheses
[INFO][2022/05/07 01:36:41 PM] Completed optimization with 969 tracks
[INFO][2022/05/07 01:36:41 PM] Ending BayesianTracker session


In [29]:
viewer=etsynseg.plot.imshow3d(np.array(ims), [peaks,miss],label_lay=[np.array(newlls,dtype=np.uint64),np.array(lss2,dtype=np.uint64)])


Exception in callback BaseAsyncIOLoop._handle_events(11044, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(11044, 1)>
Traceback (most recent call last):
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [ ]:

def nms(image,hitpeak,returnmask=True):
    # if clabs is None:
    #     clabs=list(np.unique(hitpeak))
    # if 0 in clabs:
    #     clabs.remove(0)
    indexs=np.argwhere(hitpeak>0)
    indexs=list(indexs)
    values=image[tuple(zip(*indexs))]
    vlabs=hitpeak[tuple(zip(*indexs))]
    zipped_pairs = zip(values,vlabs,indexs)
    #sorted_pairs = sorted(zipped_pairs,lambda x :x[2])
    zipped_pairs = list(zip(vlabs,values,indexs))
    zipped_pairs.sort(key=lambda x:(x[0],-x[1]))
    nmspoints=[]
    viewedlabs=[]
    for lab,value,ind in zipped_pairs:
        if lab in viewedlabs: continue
        else:
            viewedlabs.append(lab)
            nmspoints.append(ind)
    if returnmask:
        mask=np.zeros_like(image)
        mask[tuple(zip(*nmspoints))]=1
        return mask
    else:
        return nmspoints

def keep_lable_seg(segment,labs):
    ls=np.zeros_like(segment)
    for lab in labs:
        ls[segment==lab]=lab
    return ls


def track_by_seg(image,segment,
                 radius,num_iter,searchbox,
                 keep_time=3,resvers=None,labcounts={},
                sizeth=np.inf,method="geo",adth=None,
                smoothing=0,lambda1=1, lambda2=3):
    if segment is None:
        return
    segment=segment.copy()
    if resvers is not None:
        segment[segment==0]=resvers[segment==0]
        
    adth=local_threshold(image)
    peakmask=peakfilter(image,radius)*adth
    hitpeak=segment*peakmask
    
    prelab=set(np.unique(segment))
    curlab=set(np.unique(hitpeak))
    
    #intersec
    interseclab=prelab&curlab
    
    #nmsmask=nms(image,hitpeak,returnmask=True)*hitpeak
    #print(np.unique(nmsmask))
    intersecls=seg.segment.foreach_grow_area(image,num_iter,hitpeak,
                     searchbox=searchbox,sizeth=sizeth,
                     adth=adth,method=method,smoothing=smoothing,
                     lambda1=lambda1,lambda2=lambda2)
        
    #diff # which not find
    diflab=prelab-curlab
    nofinders=keep_lable_seg(segment,diflab)
    for lab in diflab:
        if lab not in labcounts:
            labcounts[lab]=1
        elif labcounts[lab]>keep_time:
            nofinders[nofinders==lab]=0
        else:
            labcounts[lab]+=1
    
    return intersecls,peakmask,nofinders